## 3-2-2 EDA_Linearity & Multicollinearity

In [1]:
# Import the necessary libraries
import os 
import pandas as pd
import numpy as np
from timezonefinder import TimezoneFinder
import matplotlib.pyplot as plt

import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor

### Load the datasets

In [2]:
# Define a list of states for which datasets will be loaded
state_list = ['Texas', 'Iowa', 'Nevada', 'Seattle']

# Define the base folder path where the datasets are located
base_folder_path = './Data/'

# Create an empty dictionary to store DataFrames
dfs = {}

# Loop through each state and load the corresponding dataset
for state in state_list:
    # Construct the file path for the dataset of the current state
    file_path = os.path.join(base_folder_path, f'{state}_0_18.csv')
    
    # Define a key for the DataFrame in the dictionary
    df_key = f'df_{state}'
    
    # Load the dataset into a Pandas DataFrame and store it in the dictionary
    dfs[df_key] = pd.read_csv(file_path)

# List the keys (DataFrame names) stored in the 'dfs' dictionary
loaded_datasets = list(dfs)
print(loaded_datasets)

['df_Texas', 'df_Iowa', 'df_Nevada', 'df_Seattle']


### Convert the datetime from UTC to LST (Local Standard Time)

In [3]:
# Define a dictionary that maps state names to their corresponding latitude and longitude coordinates
locations = {
    'Texas': (30.6095651015, -96.3402969602),
    'Nevada': (36.089758655, -115.1833049303),
    'Iowa': (42.0135891695, -93.6356966723),
    'Seattle': (48.6108182606, -123.1479221551)
}

# Initialize a TimezoneFinder instance to determine timezones based on coordinates
tf = TimezoneFinder()

# Assuming 'dfs' is your dictionary of DataFrames
for df_key in dfs:
    # Extract the state name from the DataFrame key
    state = df_key.split('_')[1]

    # Retrieve the latitude and longitude for the state from the 'locations' dictionary
    latitude, longitude = locations[state]

    # Find the timezone for the given latitude and longitude using the TimezoneFinder
    local_timezone = tf.timezone_at(lat=latitude, lng=longitude)

    # Convert the 'datetime' column in the DataFrame to datetime objects
    dfs[df_key]['datetime'] = pd.to_datetime(dfs[df_key]['datetime'])

    # Set the timezone of the 'datetime' column to UTC
    dfs[df_key]['datetime'] = dfs[df_key]['datetime'].dt.tz_localize('UTC')

    # Convert the 'datetime' column to the local timezone
    dfs[df_key]['datetime'] = dfs[df_key]['datetime'].dt.tz_convert(local_timezone)
    dfs[df_key]['datetime'] = dfs[df_key]['datetime'].dt.tz_localize(None)

In [4]:
dfs['df_Texas'].head()

,datetime,ta,GHI,rh,va,cloud,forecast,WBGT
0,2023-05-31 19:00:00,30.78134,199.9,46.2,2.506479,0.0,0,25.638261
1,2023-05-31 20:00:00,29.45638,23.1,49.9,1.875911,4.0,0,23.810995
2,2023-05-31 20:00:00,28.69420,28.2,52.6,3.799318,0.0,1,23.394587
3,2023-05-31 21:00:00,27.83575,0.0,55.4,3.317077,0.0,0,22.780433
4,2023-05-31 21:00:00,26.84966,0.0,65.8,4.094820,0.0,1,23.055983


### Extract only hour of 12 and 15 and forecast of 0 and 3

In [5]:
# Iterate through each DataFrame in the dictionary
for df_key in dfs:
    # Convert the 'datetime' column to pandas datetime format
    dfs[df_key]['datetime'] = pd.to_datetime(dfs[df_key]['datetime'])

    # Define the desired times for filtering
    desired_times = ['12:00:00', '15:00:00']

    # Filter the DataFrame for hours between 12:00:00 and 15:00:00
    dfs[df_key] = dfs[df_key][dfs[df_key]['datetime'].dt.strftime('%H:%M:%S').isin(desired_times)]

    # Define the desired forecast values for filtering
    desired_forecasts = [0, 3]

    # Extract rows where forecast is either 0 or 3
    dfs[df_key] = dfs[df_key][dfs[df_key]['forecast'].isin(desired_forecasts)]

    # Reset the index of the filtered DataFrame
    dfs[df_key] = dfs[df_key].reset_index(drop=True)

In [6]:
dfs['df_Texas'].head()

,datetime,ta,GHI,rh,va,cloud,forecast,WBGT
0,2023-06-01 12:00:00,29.74075,814.0,56.1,1.814841,37.0,0,27.844090
1,2023-06-01 12:00:00,29.84838,824.0,51.1,3.324882,18.0,3,27.233215
2,2023-06-01 15:00:00,32.01068,926.0,45.7,2.814674,36.0,0,28.431339
3,2023-06-01 15:00:00,31.94070,939.0,44.1,3.461604,24.0,3,28.113057
4,2023-06-02 12:00:00,30.55410,952.0,54.7,2.253125,21.0,0,28.313648


### Calculates differences based on the observed values (when forecast == 0)

In [7]:
# Iterate through each DataFrame in the dictionary
for df_key in dfs:
    # Convert the 'datetime' column to pandas datetime format
    dfs[df_key]['datetime'] = pd.to_datetime(dfs[df_key]['datetime'])

    # Define the desired times for filtering
    desired_times = ['12:00:00', '15:00:00']

    # Filter the DataFrame for hours between 12:00:00 and 15:00:00
    dfs[df_key] = dfs[df_key][dfs[df_key]['datetime'].dt.strftime('%H:%M:%S').isin(desired_times)]

    # Define the desired forecast values for filtering
    desired_forecasts = [0, 3]

    # Extract rows where forecast is either 0 or 3
    dfs[df_key] = dfs[df_key][dfs[df_key]['forecast'].isin(desired_forecasts)]

    # Reset the index of the filtered DataFrame
    dfs[df_key] = dfs[df_key].reset_index(drop=True)


# Iterate through each DataFrame in the dictionary
for df_key in dfs:
    # Convert the 'datetime' column to pandas datetime format
    dfs[df_key]['datetime'] = pd.to_datetime(dfs[df_key]['datetime'])

    # Filter to get the rows where forecast == 0
    reference_df = dfs[df_key][dfs[df_key]['forecast'] == 0]

    # Merge the original DataFrame with the reference DataFrame
    merged_df = dfs[df_key].merge(reference_df[['datetime', 'ta', 'GHI', 'rh', 'va', 'cloud', 'WBGT']], on='datetime', suffixes=('', '_ref'))

    # Calculate the forecast error for each column
    merged_df['ta_error'] = merged_df['ta'] - merged_df['ta_ref']
    merged_df['GHI_error'] = merged_df['GHI'] - merged_df['GHI_ref']
    merged_df['rh_error'] = merged_df['rh'] - merged_df['rh_ref']
    merged_df['va_error'] = merged_df['va'] - merged_df['va_ref']
    merged_df['cloud_error'] = merged_df['cloud'] - merged_df['cloud_ref']
    merged_df['WBGT_error'] = merged_df['WBGT'] - merged_df['WBGT_ref']

    # Save the merged DataFrame back into the dictionary
    dfs[df_key] = merged_df

In [8]:
dfs['df_Texas'].columns

Index(['datetime', 'ta', 'GHI', 'rh', 'va', 'cloud', 'forecast', 'WBGT',
       'ta_ref', 'GHI_ref', 'rh_ref', 'va_ref', 'cloud_ref', 'WBGT_ref',
       'ta_error', 'GHI_error', 'rh_error', 'va_error', 'cloud_error',
       'WBGT_error'],
      dtype='object')

In [9]:
dfs['df_Texas'].head()

,datetime,ta,GHI,rh,va,cloud,forecast,WBGT,ta_ref,GHI_ref,rh_ref,va_ref,cloud_ref,WBGT_ref,ta_error,GHI_error,rh_error,va_error,cloud_error,WBGT_error
0,2023-06-01 12:00:00,29.74075,814.0,56.1,1.814841,37.0,0,27.844090,29.74075,814.0,56.1,1.814841,37.0,27.844090,0.00000,0.0,0.0,0.000000,0.0,0.000000
1,2023-06-01 12:00:00,29.84838,824.0,51.1,3.324882,18.0,3,27.233215,29.74075,814.0,56.1,1.814841,37.0,27.844090,0.10763,10.0,-5.0,1.510041,-19.0,-0.610875
2,2023-06-01 15:00:00,32.01068,926.0,45.7,2.814674,36.0,0,28.431339,32.01068,926.0,45.7,2.814674,36.0,28.431339,0.00000,0.0,0.0,0.000000,0.0,0.000000
3,2023-06-01 15:00:00,31.94070,939.0,44.1,3.461604,24.0,3,28.113057,32.01068,926.0,45.7,2.814674,36.0,28.431339,-0.06998,13.0,-1.6,0.646930,-12.0,-0.318283
4,2023-06-02 12:00:00,30.55410,952.0,54.7,2.253125,21.0,0,28.313648,30.55410,952.0,54.7,2.253125,21.0,28.313648,0.00000,0.0,0.0,0.000000,0.0,0.000000


### Combined as one dataframe

In [10]:
#Rename each columns

df_T = dfs['df_Texas'][['datetime', 'GHI_error', 'cloud_error', 'forecast']]
df_T = df_T.rename(columns={'GHI_error': 'GHI_error_Texas', 'cloud_error': 'cloud_error_Texas'})

df_I = dfs['df_Iowa'][['datetime', 'GHI_error', 'cloud_error', 'forecast']]
df_I = df_I.rename(columns={'GHI_error': 'GHI_error_Iowa', 'cloud_error': 'cloud_error_Iowa'})

df_N = dfs['df_Nevada'][['datetime', 'GHI_error', 'cloud_error', 'forecast']]
df_N = df_N.rename(columns={'GHI_error': 'GHI_error_Nevada', 'cloud_error': 'cloud_error_Nevada'})

df_S = dfs['df_Seattle'][['datetime', 'GHI_error', 'cloud_error', 'forecast']]
df_S = df_S.rename(columns={'GHI_error': 'GHI_error_Seattle', 'cloud_error': 'cloud_error_Seattle'})

# Custom suffixes can be provided to avoid column name conflicts
custom_suffixes = ('_dfT', '_dfI')  # You can choose appropriate suffixes

# Merging df_T and df_I with custom suffixes, on both 'datetime' and 'forecast'
combined_df = pd.merge(df_T, df_I, on=['datetime', 'forecast'], how='inner', suffixes=custom_suffixes)

# Merging with df_N and df_S
# Note: You may need to adjust suffixes again if there are still conflicts
combined_df = pd.merge(combined_df, df_N, on=['datetime', 'forecast'], how='inner')
combined_df = pd.merge(combined_df, df_S, on=['datetime', 'forecast'], how='inner')

combined_df.head()

,datetime,GHI_error_Texas,cloud_error_Texas,forecast,GHI_error_Iowa,cloud_error_Iowa,GHI_error_Nevada,cloud_error_Nevada,GHI_error_Seattle,cloud_error_Seattle
0,2023-06-01 12:00:00,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0
1,2023-06-01 12:00:00,10.0,-19.0,3,78.0,4.0,13.0,0.0,1.0,-18.0
2,2023-06-01 15:00:00,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0
3,2023-06-01 15:00:00,13.0,-12.0,3,-472.0,21.0,604.0,14.0,4.0,0.0
4,2023-06-02 12:00:00,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0


In [11]:
#only 3-hour forecast
for df_key in dfs:
    dfs[df_key] = dfs[df_key].loc[dfs[df_key]['forecast'] == 3]

    # Reset the index
    dfs[df_key] = dfs[df_key].reset_index(drop=True)

# only for errors
for df_key in dfs:
    error_columns = dfs[df_key].filter(like='_error').columns
    dfs[df_key] = dfs[df_key][error_columns]

In [12]:
dfs['df_Texas'].head()

,ta_error,GHI_error,rh_error,va_error,cloud_error,WBGT_error
0,0.10763,10.0,-5.0,1.510041,-19.0,-0.610875
1,-0.06998,13.0,-1.6,0.646930,-12.0,-0.318283
2,0.10623,3.0,-7.5,0.374193,-21.0,-0.881532
3,0.07323,-101.0,-7.3,0.956350,3.0,-0.991742
4,0.82907,-1.0,-7.8,2.425207,0.0,-0.434563


### Desciptive analysis

In [13]:
dfs['df_Texas'].describe()

,ta_error,GHI_error,rh_error,va_error,cloud_error,WBGT_error
count,182.000000,182.000000,182.000000,182.000000,182.000000,182.000000
mean,0.378071,21.854475,-7.194505,0.829210,-4.489011,-0.649703
std,0.840473,161.994961,5.259936,0.817567,17.014047,0.702607
min,-2.716400,-662.000000,-35.400000,-2.382283,-88.000000,-4.311828
25%,-0.091873,-7.000000,-9.775000,0.353159,-8.000000,-1.007250
50%,0.392315,1.000000,-7.300000,0.897916,0.000000,-0.696066
75%,0.828918,8.750000,-3.700000,1.264807,0.000000,-0.409980
max,4.779540,920.000000,6.900000,4.360645,85.000000,2.990307


In [14]:
dfs['df_Iowa'].describe()

,ta_error,GHI_error,rh_error,va_error,cloud_error,WBGT_error
count,182.000000,182.000000,182.000000,182.000000,182.000000,182.000000
mean,0.475255,42.433596,-10.192857,0.306545,-8.675824,-0.632080
std,1.280092,220.802400,8.496421,0.966579,22.611769,1.246516
min,-3.311980,-847.000000,-60.800000,-1.941591,-100.000000,-4.669727
25%,-0.198453,-18.000000,-14.975000,-0.336051,-12.000000,-1.302929
50%,0.401445,0.000000,-10.050000,0.243125,-2.000000,-0.846514
75%,1.022063,57.801758,-5.250000,0.882800,0.000000,-0.407970
max,6.336700,789.300000,14.500000,4.247968,74.000000,3.936887


In [15]:
dfs['df_Nevada'].describe()

,ta_error,GHI_error,rh_error,va_error,cloud_error,WBGT_error
count,182.000000,182.000000,182.000000,182.000000,182.000000,182.000000
mean,0.896743,54.755846,-1.791758,1.038757,-0.774725,0.533254
std,1.075863,208.967537,4.154913,1.557631,14.053914,0.994369
min,-6.078980,-686.000000,-24.000000,-2.878845,-100.000000,-4.749224
25%,0.430532,-4.000000,-3.100000,0.041896,0.000000,0.152604
50%,0.980295,1.000000,-1.500000,0.842684,0.000000,0.458049
75%,1.362643,8.075000,-0.325000,1.745410,0.000000,0.759391
max,4.631960,924.000000,24.000000,8.750594,50.000000,4.308567


In [16]:
dfs['df_Seattle'].describe()

,ta_error,GHI_error,rh_error,va_error,cloud_error,WBGT_error
count,182.000000,182.000000,182.000000,182.000000,182.000000,182.000000
mean,-0.920221,26.020132,-0.486813,0.572755,-5.126374,-0.801226
std,0.927690,145.625699,6.680957,1.015183,18.546655,0.973458
min,-3.098910,-627.000000,-23.400000,-2.785736,-95.000000,-3.686593
25%,-1.477985,-7.000000,-3.325000,-0.071154,-2.000000,-1.354121
50%,-1.086930,0.650000,0.050000,0.604506,0.000000,-1.009073
75%,-0.457287,7.991000,3.725000,1.204860,0.000000,-0.517399
max,2.853540,654.700000,13.000000,2.562517,56.000000,3.119619


### VIF

In [17]:
# delette WBGT_error table
for df_key in dfs:
    dfs[df_key] = dfs[df_key].drop('WBGT_error', axis=1) # delette WBGT_error table


In [18]:
dfs['df_Seattle'].describe()

,ta_error,GHI_error,rh_error,va_error,cloud_error
count,182.000000,182.000000,182.000000,182.000000,182.000000
mean,-0.920221,26.020132,-0.486813,0.572755,-5.126374
std,0.927690,145.625699,6.680957,1.015183,18.546655
min,-3.098910,-627.000000,-23.400000,-2.785736,-95.000000
25%,-1.477985,-7.000000,-3.325000,-0.071154,-2.000000
50%,-1.086930,0.650000,0.050000,0.604506,0.000000
75%,-0.457287,7.991000,3.725000,1.204860,0.000000
max,2.853540,654.700000,13.000000,2.562517,56.000000


In [23]:
import pandas as pd
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor

# Assuming dfs is your dictionary of DataFrames for each state
vif_data_per_state = {}

for df_key, df in dfs.items():
    # Adding a constant term for intercept
    df_with_const = sm.add_constant(df)

    # Initialize DataFrame to store VIF for each variable
    vif_data = pd.DataFrame()
    vif_data['Variable'] = df_with_const.columns

    # Calculate VIF for each variable
    vif_data['VIF'] = [variance_inflation_factor(df_with_const.values, i) for i in range(df_with_const.shape[1])]

    # Store the VIF data in the dictionary with state name as the key
    vif_data_per_state[df_key] = vif_data

    # Optionally print each state's VIF data
    print(f'VIF Data for {df_key}:')
    print(vif_data)


VIF Data for df_Texas:
      Variable       VIF
0        const  5.338485
1     ta_error  4.299881
2    GHI_error  1.436303
3     rh_error  3.779457
4     va_error  1.113180
5  cloud_error  1.101588
VIF Data for df_Iowa:
      Variable       VIF
0        const  3.668454
1     ta_error  3.497294
2    GHI_error  1.680267
3     rh_error  3.020493
4     va_error  1.040592
5  cloud_error  1.297027
VIF Data for df_Nevada:
      Variable       VIF
0        const  3.225635
1     ta_error  3.881232
2    GHI_error  1.427629
3     rh_error  3.264518
4     va_error  1.196482
5  cloud_error  1.202477
VIF Data for df_Seattle:
      Variable       VIF
0        const  3.948036
1     ta_error  2.407494
2    GHI_error  1.340338
3     rh_error  2.111603
4     va_error  1.054900
5  cloud_error  1.200029


In [24]:
list(vif_data_per_state)

['df_Texas', 'df_Iowa', 'df_Nevada', 'df_Seattle']

In [25]:
vif_data_per_state['df_Texas']

,Variable,VIF
0,const,5.338485
1,ta_error,4.299881
2,GHI_error,1.436303
3,rh_error,3.779457
4,va_error,1.113180
5,cloud_error,1.101588


In [26]:
vif_data_per_state['df_Iowa']

,Variable,VIF
0,const,3.668454
1,ta_error,3.497294
2,GHI_error,1.680267
3,rh_error,3.020493
4,va_error,1.040592
5,cloud_error,1.297027


In [27]:
vif_data_per_state['df_Nevada']

,Variable,VIF
0,const,3.225635
1,ta_error,3.881232
2,GHI_error,1.427629
3,rh_error,3.264518
4,va_error,1.196482
5,cloud_error,1.202477


In [28]:
vif_data_per_state['df_Seattle']

,Variable,VIF
0,const,3.948036
1,ta_error,2.407494
2,GHI_error,1.340338
3,rh_error,2.111603
4,va_error,1.054900
5,cloud_error,1.200029
